In [ ]:
from sqlalchemy.engine import result
from requests.models import Response
import  requests
import re
import urllib
from urllib.parse import urlparse
# from tld import get_tld
from bs4 import BeautifulSoup
# import whois
import socket
import ipaddress
from datetime import datetime

class url_data:
    def __init__(self,url):
        self.url=url
#function to check the ip address
def have_ip(url):
  try:
    ipaddress.ip_address(url)
    ip=1
  except:
    ip=0
  return ip

#function to check At @
def have_at(url):
  return 1 if '@' in url else 0

#function to check length of the url
def length_url(url):
  return 1 if len(url)>54 else 0

#function to check url_depth
def url_depth(url):
  return url.count('/')-2
  # s = url.split('/')
  # depth = 0
  # for i in range(len(s)):
  #   if len(s[i]) != 0:
  #     depth+=1
  #   return depth

#function to check the redirection
def redirection(url):
  c=0
  for i in range(len(url)):
    if url[i]=='/' and url[i+1]=='/':
      c+=1
      break
    else:
      c+=1
  return 1 if c>6 else 0
  # pos = url.rfind('//')
  # if pos > 6:
  #   if pos > 7:
  #     return 1
  #   else:
  #     return 0
  # else:
  #   return 0

#function to check the https domain
def httpsDomain(url):
  domain = urlparse(url).netloc
  return 1 if 'https' in domain else 0

#function to check sortening services
shortening = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                      r"tr\.im|link\.zip\.net"
def sort_url(url):
  sort=re.search(shortening,url)
  if sort:
    return 2
  else:
    return 0

#function from prefix and suffix
def prefix_suffix(url):
  if '-' in urlparse(url).netloc:
    return 1
  else:
    return 0

#function to find web trafic
def web_traffic(url):
  try:
    #Filling the whitespaces in the URL if any
    response = requests.get(f"https://www.alexa.com/siteinfo/{url}")
    response.raise_for_status()  # Check for any request errors
    soup = BeautifulSoup(response.content, 'html.parser')
    rank = soup.find("strong", class_="metrics-data align-vmiddle")
    # url = urllib.parse.quote(url)
    # rank = BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find(
    #     "REACH")['RANK']
    rank = int(rank)
  except TypeError:
        return 1
  if rank <100000:
    return 1
  else:
    return 0

#function to find domain age
def age(domain_name):
  creation_date = domain_name.creation_date
  expiration_date = domain_name.expiration_date
  if (isinstance(creation_date,str) or isinstance(expiration_date,str)):
    try:
      creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
      expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
    except:
      return 1
  if ((expiration_date is None) or (creation_date is None)):
    return 1
  elif ((type(expiration_date) is list) or (type(creation_date) is list)):
    return 1
  else:
    ageofdomain = abs((expiration_date - creation_date).days)
    if ((ageofdomain/30) < 6):
      age = 1
    else:
        age = 0
  return age

#function to find the domain end
def domain_end(domain_name):
  expiration_date = domain_name.expiration_date
  if isinstance(expiration_date,str):
    try:
      expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
    except:
      return 1
  if (expiration_date is None):
      return 1
  elif (type(expiration_date) is list):
      return 1
  else:
    today = datetime.now()
    end = abs((expiration_date - today).days)
    if ((end/30) < 6):
      end = 0
    else:
      end = 1
  return end

#function to find the Iframe
def Iframe(response):
  if response=="":
    return 1
  else:
    if re.findall(r"[<iframe>|<frameBorder>]", response.text):
      return 0
    else:
      return 1

#function to find the mouse over the status bar
def mouse_over(response):
  if response == "" :
    return 1
  else:
    if re.findall("<script>.+onmouseover.+</script>", response.text):
      return 1
    else:
      return 0

#function to check the right click attributes
def right_click(response):
  if response == "":
    return 1
  else:
    if re.findall(r"event.button ?== ?2", response.text):
      return 0
    else:
      return 1

#function for finding web_forwarding
def forward(response):
  if response == "":
    return 1
  else:
    if len(response.history) <= 2:
      return 0
    else:
      return 1

#main data's append in the list
url=input()
#creating empty list to store data
data_url=[]
#inserting the data_url to the list
data_url.append(have_ip(url))
data_url.append(have_at(url))
data_url.append(length_url(url))
data_url.append(url_depth(url))
data_url.append(redirection(url))
data_url.append(httpsDomain(url))
data_url.append(sort_url(url))
data_url.append(prefix_suffix(url))
#finding the domain name
dns = 0
try:
  domain_name = whois.whois(urlparse(url).netloc)
except:
  dns = 1
data_url.append(dns)
data_url.append(web_traffic(url))
data_url.append(1 if dns == 1 else age(domain_name))
data_url.append(1 if dns == 1 else domain_end(domain_name))
#geting the response from the url
try:
  response=requests.get(url)
except:
  response=""
#inserting the informaition based on the response
data_url.append(Iframe(response))
data_url.append(mouse_over(response))
data_url.append(right_click(response))
data_url.append(forward(response))
url_predict_data=[]
url_predict_data.append(data_url)
print(url_predict_data)


# #checking the feature improtance in the model
# plt.figure(figsize=(9,7))
# n_features = X_train.shape[1]
# plt.barh(range(n_features), tree.feature_importances_, align='center')
# plt.yticks(np.arange(n_features), X_train.columns)
# plt.xlabel("Feature importance")
# plt.ylabel("Feature")
# plt.show()


[[0, 0, 1, 2, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0]]

https://ltqpp.research.google.com/drive/1vwjmMsSW8pFrQgYipbTdmoD9FCzBW7W_
[[0, 0, 1, 2, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]]


[[0, 0, 1, 2, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0]]

In [ ]:

# machine learning
#importing llb
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pickle
#load dataset
df = pd.read_csv('dataset.csv')
#Dropping the Domain column
data = df.drop(['Domain'], axis = 1).copy()
# shuffling the data in the dataset
data = data.sample(frac=1).reset_index(drop=True)
# Sepratating & assigning features and target columns to X & y
#input data from the data set
y = data['Label']
X = data.drop('Label',axis=1)
# Splitting the dataset into train and test sets: 80-20 split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
# Creating holders to store the model performance results
ML_Model = []
acc_train = []
acc_test = []
#function to call for storing the results
def storeResults(model, a,b):
  ML_Model.append(model)
  acc_train.append(round(a, 3))
  acc_test.append(round(b, 3))
# machine learning alg
from sklearn.tree import DecisionTreeClassifier
url_predict_data_df = pd.DataFrame(columns=X_train.columns)
# Populate the DataFrame with feature values for the URL you want to predict
url_predict_data_df.loc[0] = data_url
# instantiate the model
tree = DecisionTreeClassifier(max_depth = 5)
# fit the model
tree.fit(X_train, y_train)
#predicting the target value from the model for the samples
y_test_tree = tree.predict(X_test)
y_train_tree = tree.predict(X_train)
acc_train_tree = accuracy_score(y_train,y_train_tree)
acc_test_tree = accuracy_score(y_test,y_test_tree)
prediction=tree.predict(url_predict_data_df)
print("Phishing" if prediction[0]==1 else "Legitimate")
print("acc-train",acc_train_tree)
print("acc-test",acc_test_tree)


#Plotting the data distribution
df.hist(bins = 50,figsize = (15,15))
plt.show()


#Correlation heatmap

plt.figure(figsize=(10,7))
sns.heatmap(df.corr())
plt.show()
